In [3]:
import os
import numpy as np
import pandas as pd
from skimage.io import imread, imsave
from skimage.filters import threshold_otsu
from skimage.measure import label, regionprops
from skimage.color import label2rgb
from skimage.segmentation import mark_boundaries
import matplotlib.pyplot as plt
from pathlib import Path
from IPython.display import display

def process_image(image):
    if image.ndim == 3:
        image = image[..., 0]  # Convert to grayscale if RGB
    thresh = threshold_otsu(image)
    mask_coacervate = image > thresh
    mask_background = ~mask_coacervate

    Iav = image[mask_coacervate].mean() if np.any(mask_coacervate) else 0
    Ibk = image[mask_background].mean() if np.any(mask_background) else 0

    return Iav, Ibk, mask_coacervate

def analyze_images():
    folder = Path("/content")
    image_files = sorted([f for f in folder.glob("*.tif") if f.stem != "blank"])
    blank_file = folder / "blank.tif"

    if not blank_file.exists():
        raise FileNotFoundError("Missing 'blank.tif' in the folder")

    blank_image = imread(blank_file)
    if blank_image.ndim == 3:
        blank_image = blank_image[..., 0]
    Ib = blank_image.mean()

    results = []

    for file in image_files:
        img = imread(file)
        if img.ndim == 3:
            img = img[..., 0]

        Iav, Ibk, mask = process_image(img)

        # Label and extract region properties
        labeled = label(mask)
        regions = regionprops(labeled, intensity_image=img)

        intensities = []
        for i, region in enumerate(regions):
            if region.area >= 5:  # optional: filter small noise
                intensities.append({"Droplet_ID": i + 1, "Mean_Intensity": region.mean_intensity})

        df_droplets = pd.DataFrame(intensities)
        droplet_csv_path = folder / f"{file.stem}_droplets.csv"
        df_droplets.loc[len(df_droplets.index)] = {"Droplet_ID": "Average", "Mean_Intensity": df_droplets["Mean_Intensity"].mean()}
        df_droplets.to_csv(droplet_csv_path, index=False)

        # Overlay boundaries
        overlay = mark_boundaries(img, labeled, color=(1, 0, 0))
        fig, ax = plt.subplots(figsize=(6, 6))
        ax.imshow(overlay, cmap="gray")
        ax.set_title(f"Detected Droplets: {file.name}")
        ax.axis("off")
        plot_path = folder / f"{file.stem}_droplets_overlay.png"
        plt.savefig(plot_path, bbox_inches='tight')
        plt.close()

# Run the function
analyze_images()
